## TreeHarmonizer

This notebook serves to place SVs called by Severus onto the given phylogenetic tree.

This notebook is being converted to serve as a element of a single TreeHarmonizer tool.

### Dependencies

----- Please consult the main project README.md for installation instructions -----

TreeHarmonizer requires a Python 3.6 environment with the following packages -
* pandas
* io
* functools 
* os
* intervaltree (https://pypi.org/project/intervaltree/)
* ete3
* jupyter

## How To Run

1. Load Dependencies in the cell below
2. Fill in input parameters per instructions
3. Run all following cells

In [1]:
import importlib
import pandas as pd
import TreeHarmonizer_utils as th_utils
importlib.reload(th_utils)
import subprocess

pd.set_option('display.width', 5000)
pd.set_option("display.expand_frame_repr", True)
pd.set_option("display.max_colwidth", 1000)
pd.set_option('display.max_columns', None)

## Input Parameters

### SV Path
* Expects a direct path to the to the Severus output VCF of all somatic SVs.

In [16]:
# Modify paths with your file structures below.
# Paths are prepopulated for example data provided in the repository. 
# Relative paths get converted to absolute paths via os.path.abspath in the th_utils functions.

sv_path = "./example_data/sv/severus_chr1.vcf"
fn_rate = 0.15
write_exclusive_vcfs = True
write_cumulative_vcfs = True
write_path_exclusive = "./severus_output_vcfs/exclusive"
write_path_cumulative = "./severus_output_vcfs/cumulative"

## Load severus and tree data

In [17]:
# Load Severus data into a merged DataFrame
severus = th_utils.generate_severus_df(severus_path=sv_path, simple_name=True)

# Load tree input via newick string and parse it into various components
imported_tree, non_terminals, terminals, non_terminal_paths, terminal_paths, non_terminal_leaves, terminal_paths_o_keys, non_terminal_paths_without_N1 = th_utils.get_tree_data()

## Filter out sex chromosomes and filter out second BND pair for no double counting the same SV

In [18]:
# Import severus data

sev_df = pd.DataFrame()
sev_df_no_vntr = pd.DataFrame()

# Set CHROM column to type string
severus['CHROM'] = severus['CHROM'].astype(str)

sev = [severus]
for x in range(len(sev)):
    #print("Data type: " + str(data_type) + " | Data set: " + str(x))

    # Remove severus data with X or Y endpoints as sex chromosomes are not in use
    data = sev[x]
    sev_x_or_y_endpoint = data[ (data['ALT'].str.contains("X:") == True) | (data['ALT'].str.contains("Y:") == True)].copy()
    ids_to_remove = sev_x_or_y_endpoint['ID'].tolist()
    id_pairs_to_remove = [x[:-1]+"2" for x in ids_to_remove]
    all_ids_to_remove = ids_to_remove + id_pairs_to_remove
    data = data[~data['ID'].isin(all_ids_to_remove)].copy()
    
    # Remove second breakend pair of each BND type variant
    bnd_second_breakends = data[ (data['ID'].str.contains("BND") == True) & (data['ID'].str.endswith("2") == True)].copy()
    data = data[~data['ID'].isin(bnd_second_breakends['ID'])].copy()

    #print(all_ids_to_remove)
    sev[x] = data.copy(deep=True)

sev_df = sev[0]

## Preprocess common ancestors

In [19]:
def severus_called_sublines_helper(row):
    # GT:VAF:hVAF:DR:DV
    # A VCF BUG IN THE CURRENT VERSION IS CAUSING THE ORIGINAL FILTER TO BREAK
    # return [col for col in severus.columns if col.startswith('C') and col != "CHROM" and row[col] != "./.:0:0,0,0:0:0"]
    # FILTER HAS BEEN CHANGED TO THE FOLLOWING
    output_subline_list = []
    for col in severus.columns:
        if col.startswith('C') and col != "CHROM":
            internal_sev_data = row[col].split(":")
            DV = int(internal_sev_data[4])
            if DV > 0:
                output_subline_list.append(col)
    return output_subline_list


sev_df['severus_called_sublines'] = sev_df.apply(lambda row: severus_called_sublines_helper(row), axis=1)
sev_df['severus_mrca'] = sev_df.apply(lambda row: th_utils.common_ancestor_helper(row, "severus_called_sublines", imported_tree), axis=1)
sev_df['severus_mrca_terminals'] = sev_df.apply(lambda row: imported_tree.search_nodes(name=row['severus_mrca'])[0].get_leaf_names(), axis=1)

## Determine clade size acceptance requirements

In [20]:
# Prebuilt minimum subline support per clade size requirement
# Uncomment below in order to use the prebuilt version rather than unnecessarily recalculating

# minimum_subline_support_per_clade_size_requirement = {
#     1: 1,
#     2: 2,
#     3: 2,
#     4: 3,
#     5: 4,
#     7: 5,
#     8: 6,
#     12: 10,
#     16: 13,
#     23: 19
# }

# If other clade sizes are necessary, formulaic version of threshold is below, commented out.

clade_sizes = set()
# Get a list of all the clade sizes in the tree
for clade in imported_tree.traverse():
    if clade.is_leaf():
        continue
    clade_sizes.add(len(clade.get_leaves()))

formulaic_subline_support_per_clade_size_requirement = {}

cur_tree_clade_sizes = clade_sizes
cur_tree_fn_rate = fn_rate # Default 15% as set in Cell 2, can be changed based on user preference
for clade_size in cur_tree_clade_sizes:
    if clade_size < 2:
        formulaic_subline_support_per_clade_size_requirement[clade_size] = 1
    elif clade_size == 2:
        formulaic_subline_support_per_clade_size_requirement[clade_size] = 2
    else:
        # Calculate the support requirement based on the FN rate
        support_requirement = int(clade_size * (1 - float(cur_tree_fn_rate)))
        formulaic_subline_support_per_clade_size_requirement[clade_size] = support_requirement

minimum_subline_support_per_clade_size_requirement = formulaic_subline_support_per_clade_size_requirement

## Determine amount of placed variants

In [21]:
sev_df['called_subline_count'] = sev_df.apply(lambda row: len(row['severus_called_sublines']), axis=1)
sev_df['terminal_count'] = sev_df.apply(lambda row: len(row['severus_mrca_terminals']), axis=1)
sev_df['minimum_subline_support_threshold_met_severus'] = sev_df.apply(lambda row: row['called_subline_count'] >= minimum_subline_support_per_clade_size_requirement[row['terminal_count']], axis=1)

print(sev_df['minimum_subline_support_threshold_met_severus'].value_counts())

True     53
False     1
Name: minimum_subline_support_threshold_met_severus, dtype: int64


## Default Severus Header

In [22]:
default_severus_header = """##fileformat=VCFv4.2
##source=Severus_v1.3
##CommandLine= --target-bam C1.bam C3.bam C4.bam C5.bam C6.bam C7.bam C8.bam C9.bam C10.bam C11.bam C12.bam C13.bam C14.bam C15.bam C16.bam C17.bam C18.bam C19.bam C20.bam C21.bam C22.bam C23.bam C24.bam --control-bam normal.bam --vntr-bed /data/keskusa2/Severus/vntrs/grcm38.trf.bed --out-dir /data/KolmogorovLab/keskusa2/nano_sv_calls/mouse_results/severus_pon_Mar11 -t 32 --resolve-overlaps --phasing-vcf /data/KolmogorovLab/agoretsky/margin_whatshap_output/output.phased.vcf.gz --low-quality --PON /data/KolmogorovLab/keskusa2/FVB_PON.tsv
##fileDate=2025-03-13
##contig=<ID=1,length=195471971>
##contig=<ID=10,length=130694993>
##contig=<ID=11,length=122082543>
##contig=<ID=12,length=120129022>
##contig=<ID=13,length=120421639>
##contig=<ID=14,length=124902244>
##contig=<ID=15,length=104043685>
##contig=<ID=16,length=98207768>
##contig=<ID=17,length=94987271>
##contig=<ID=18,length=90702639>
##contig=<ID=19,length=61431566>
##contig=<ID=2,length=182113224>
##contig=<ID=3,length=160039680>
##contig=<ID=4,length=156508116>
##contig=<ID=5,length=151834684>
##contig=<ID=6,length=149736546>
##contig=<ID=7,length=145441459>
##contig=<ID=8,length=129401213>
##contig=<ID=9,length=124595110>
##contig=<ID=MT,length=16299>
##contig=<ID=X,length=171031299>
##contig=<ID=Y,length=91744698>
##contig=<ID=JH584299.1,length=953012>
##contig=<ID=GL456233.1,length=336933>
##contig=<ID=JH584301.1,length=259875>
##contig=<ID=GL456211.1,length=241735>
##contig=<ID=GL456350.1,length=227966>
##contig=<ID=JH584293.1,length=207968>
##contig=<ID=GL456221.1,length=206961>
##contig=<ID=JH584297.1,length=205776>
##contig=<ID=JH584296.1,length=199368>
##contig=<ID=GL456354.1,length=195993>
##contig=<ID=JH584294.1,length=191905>
##contig=<ID=JH584298.1,length=184189>
##contig=<ID=JH584300.1,length=182347>
##contig=<ID=GL456219.1,length=175968>
##contig=<ID=GL456210.1,length=169725>
##contig=<ID=JH584303.1,length=158099>
##contig=<ID=JH584302.1,length=155838>
##contig=<ID=GL456212.1,length=153618>
##contig=<ID=JH584304.1,length=114452>
##contig=<ID=GL456379.1,length=72385>
##contig=<ID=GL456216.1,length=66673>
##contig=<ID=GL456393.1,length=55711>
##contig=<ID=GL456366.1,length=47073>
##contig=<ID=GL456367.1,length=42057>
##contig=<ID=GL456239.1,length=40056>
##contig=<ID=GL456213.1,length=39340>
##contig=<ID=GL456383.1,length=38659>
##contig=<ID=GL456385.1,length=35240>
##contig=<ID=GL456360.1,length=31704>
##contig=<ID=GL456378.1,length=31602>
##contig=<ID=GL456389.1,length=28772>
##contig=<ID=GL456372.1,length=28664>
##contig=<ID=GL456370.1,length=26764>
##contig=<ID=GL456381.1,length=25871>
##contig=<ID=GL456387.1,length=24685>
##contig=<ID=GL456390.1,length=24668>
##contig=<ID=GL456394.1,length=24323>
##contig=<ID=GL456392.1,length=23629>
##contig=<ID=GL456382.1,length=23158>
##contig=<ID=GL456359.1,length=22974>
##contig=<ID=GL456396.1,length=21240>
##contig=<ID=GL456368.1,length=20208>
##contig=<ID=JH584292.1,length=14945>
##contig=<ID=JH584295.1,length=1976>
##ALT=<ID=DEL,Description="Deletion">
##ALT=<ID=INS,Description="Insertion">
##ALT=<ID=DUP,Description="Duplication">
##ALT=<ID=INV,Description="Reciprocal Inversion">
##ALT=<ID=BND,Description="Breakend">
##FILTER=<ID=PASS,Description="All filters passed">
##FILTER=<ID=FAIL_LOWSUPP,Description="Less number of support, but ok in other samples">
##FILTER=<ID=FAIL_MAP_CONS,Description="Majority of variant reads have unreliable mappability">
##FILTER=<ID=FAIL_CONN_CONS,Description="Majority of variant reads have unreliable connections">
##FILTER=<ID=FAIL_LOWCOV_OTHER,Description="Low variant coverage in other samples">
##INFO=<ID=PRECISE,Number=0,Type=Flag,Description="SV with precise breakpoints coordinates and length">
##INFO=<ID=IMPRECISE,Number=0,Type=Flag,Description="SV with imprecise breakpoints coordinates and length">
##INFO=<ID=SVTYPE,Number=1,Type=String,Description="Type of structural variant">
##INFO=<ID=SVLEN,Number=1,Type=Integer,Description="Length of the SV">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the SV">
##INFO=<ID=STRANDS,Number=1,Type=String,Description="Breakpoint strandedness">
##INFO=<ID=DETAILED_TYPE,Number=1,Type=String,Description="Detailed type of the SV">
##INFO=<ID=INSLEN,Number=1,Type=Integer,Description="Length of the unmapped sequence between breakpoint">
##INFO=<ID=MAPQ,Number=1,Type=Integer,Description="Median mapping quality of supporting reads">
##INFO=<ID=PHASESETID,Number=1,Type=String,Description="Matching phaseset ID for phased SVs">
##INFO=<ID=HP,Number=1,Type=Integer,Description="Matching haplotype ID for phased SVs">
##INFO=<ID=CLUSTERID,Number=1,Type=String,Description="Cluster ID in breakpoint_graph">
##INFO=<ID=INSSEQ,Number=1,Type=String,Description="Insertion sequence between breakpoints">
##INFO=<ID=MATE_ID,Number=1,Type=String,Description="MATE ID for breakends">
##INFO=<ID=INSIDE_VNTR,Number=1,Type=String,Description="True if an indel is inside a VNTR">
##INFO=<ID=ALIGNED_POS,Number=1,Type=String,Description="Position in the reference">
##INFO=<ID=LOW_COV_IN,Number=1,Type=String,Description="Samples that has low coverage in that region">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=DR,Number=1,Type=Integer,Description="Number of reference reads">
##FORMAT=<ID=DV,Number=1,Type=Integer,Description="Number of variant reads">
##FORMAT=<ID=VAF,Number=1,Type=Float,Description="Variant allele frequency">
##FORMAT=<ID=hVAF,Number=3,Type=Float,Description="Haplotype specific variant Allele frequency (H0,H1,H2)">
"""

## Generate exclusive variant placement VCFs

In [23]:
def generate_exclusive_presence_absence_vcf(internal_node, input_merged_df):
    df_filtered = input_merged_df[ (input_merged_df['minimum_subline_support_threshold_met_severus'] == True) & (input_merged_df['severus_mrca'] == internal_node) ]
    return df_filtered.copy(deep=True)

In [24]:
exclusive_dfs = {}

# Generate Exclusive Presence Absence VCFs for every node
for key, value in non_terminal_leaves.items():
    framework_df_exclusive = generate_exclusive_presence_absence_vcf(key, sev_df)
    reordered_df = framework_df_exclusive[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']]
    print("Severus Count for node: " + str(key) + " " + str(len(framework_df_exclusive)))

    if write_exclusive_vcfs:
        path_prefix = write_path_exclusive.strip("/")
        subprocess.run(['mkdir', '-p', path_prefix])
        th_utils.write_vcf(reordered_df, f"{path_prefix}/{key}.vcf", default_severus_header)

    exclusive_dfs.update({key: framework_df_exclusive})

Severus Count for node: N1 5
Severus Count for node: N8 0
Severus Count for node: N2 0
Severus Count for node: N12 0
Severus Count for node: N9 0
Severus Count for node: N4 0
Severus Count for node: N3 0
Severus Count for node: N16 0
Severus Count for node: N13 0
Severus Count for node: N10 0
Severus Count for node: O5 3
Severus Count for node: O23 4
Severus Count for node: N5 0
Severus Count for node: O19 4
Severus Count for node: O17 1
Severus Count for node: N17 0
Severus Count for node: O13 2
Severus Count for node: N14 0
Severus Count for node: O9 1
Severus Count for node: N11 0
Severus Count for node: O4 1
Severus Count for node: N6 0
Severus Count for node: N7 1
Severus Count for node: N19 0
Severus Count for node: N18 1
Severus Count for node: N15 0
Severus Count for node: O24 0
Severus Count for node: O1 1
Severus Count for node: O22 2
Severus Count for node: O10 0
Severus Count for node: O12 1
Severus Count for node: O3 2
Severus Count for node: O14 2
Severus Count for node: 

## Generate cumulative variant placement VCFs

In [25]:
cumulative_severus_dfs = {}

for key, value in non_terminal_paths.items():
    merged_for_key = pd.concat([exclusive_dfs[x] for x in value], ignore_index=True)
    
    cumulative_severus_dfs.update({key: merged_for_key})
    print("Severus Count for node: " + str(key) + " " + str(len(merged_for_key)))

    if write_cumulative_vcfs:
        path_prefix = write_path_cumulative.strip("/")
        subprocess.run(['mkdir', '-p', path_prefix])
        th_utils.write_vcf(merged_for_key, f"{path_prefix}/{key}.vcf", default_severus_header)


Severus Count for node: N1 5
Severus Count for node: N8 5
Severus Count for node: N2 5
Severus Count for node: N12 5
Severus Count for node: N9 5
Severus Count for node: N4 5
Severus Count for node: N3 5
Severus Count for node: N16 5
Severus Count for node: N13 5
Severus Count for node: N10 5
Severus Count for node: O5 8
Severus Count for node: O23 9
Severus Count for node: N5 5
Severus Count for node: O19 9
Severus Count for node: O17 6
Severus Count for node: N17 5
Severus Count for node: O13 7
Severus Count for node: N14 5
Severus Count for node: O9 6
Severus Count for node: N11 5
Severus Count for node: O4 6
Severus Count for node: N6 5
Severus Count for node: N7 6
Severus Count for node: N19 5
Severus Count for node: N18 6
Severus Count for node: N15 5
Severus Count for node: O24 5
Severus Count for node: O1 6
Severus Count for node: O22 7
Severus Count for node: O10 5
Severus Count for node: O12 6
Severus Count for node: O3 8
Severus Count for node: O14 8
Severus Count for node: 